In [12]:
! pip install pyspark

# Formato LIBSVM:

<mark>def:</mark> Biblioteca de ML Open Source desenvolvida pela National Taiwan University.

# Matriz densa x esparsa:
todos valores da matriz são informados > <mark> densa </mark> <br>
apenas valores diferentes de zero da matriz são informados > <mark> esparsa </mark> 
(dados otimizados em proc e arm) - deve apresentar alguns elementos de controle 
(n basta eliminar somente os zeros) - Exemplo: LIBSVM


In [14]:
#imports

import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init() # inicializa interface pyspark jupyter
spark = SparkSession.builder.appName("importacao").getOrCreate()
#inicializa sessão no spark com nome importação

In [16]:
#definindo schema:
arqschema = "id int, nome String, status String, cidade String, vendas int, data Date"
despachantes = spark.read.csv("despachantes.csv", header=False, schema=arqschema)
despachantes.show(5)

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
+---+-------------------+------+-------------+------+----------+
only showing top 5 rows



In [17]:
#testando se schema deu certo:
despachantes.schema

StructType([StructField('id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', DateType(), True)])

In [23]:
#Inferindo schema sozinho: (load importa vários formatos, porém é necessário definí-lo)
desp_auto_schema = spark.read.load("despachantes.csv", format="csv", header=False, inferSchema=True, sep="," )
desp_auto_schema.show(5)

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
+---+-------------------+-----+-------------+---+----------+
only showing top 5 rows



In [25]:
#testando se schema deu certo:
desp_auto_schema.schema

StructType([StructField('_c0', IntegerType(), True), StructField('_c1', StringType(), True), StructField('_c2', StringType(), True), StructField('_c3', StringType(), True), StructField('_c4', IntegerType(), True), StructField('_c5', DateType(), True)])

In [33]:
# Formato Parquet: (tem propriedade de suportar schema, como no json e orc - default do spark >
# mais rápido em termos de leitura)
par = spark.read.format("parquet").load("despachantes.parquet")
par.show()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [28]:
# Formato json (colunas definidas no arquivo):
js = spark.read.format("json").load("despachantes.json")
js.show()

+-------------+-----------+---+-------------------+------+------+
|       cidade|       data| id|               nome|status|vendas|
+-------------+-----------+---+-------------------+------+------+
|  Santa Maria| 2020-08-11|  1|   Carminda Pestana| Ativo|    23|
|Novo Hamburgo| 2020-03-05|  2|    Deolinda Vilela| Ativo|    34|
| Porto Alegre| 2020-02-05|  3|   Emídio Dornelles| Ativo|    34|
| Porto Alegre| 2020-02-05|  4|Felisbela Dornelles| Ativo|    36|
| Porto Alegre| 2020-02-05|  5|     Graça Ornellas| Ativo|    12|
| Porto Alegre| 2019-01-05|  6|   Matilde Rebouças| Ativo|    22|
|  Santa Maria| 2019-10-05|  7|    Noêmia   Orriça| Ativo|    45|
| Porto Alegre| 2020-03-05|  8|      Roque Vásquez| Ativo|    65|
| Porto Alegre| 2018-05-05|  9|      Uriel Queiroz| Ativo|    54|
| Porto Alegre| 2020-09-05| 10|   Viviana Sequeira| Ativo|     0|
+-------------+-----------+---+-------------------+------+------+



In [31]:
# Formato Orc (padrão big data, é colunar > mais compactado comparando ao pqt)
orc = spark.read.format("orc").load("despachantes.orc")
orc.show()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+

